# 📄 Advanced Report Customization

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Notebook Information</b><br>
<b>Level:</b> Advanced<br>
<b>Estimated Time:</b> 20 minutes<br>
<b>Prerequisites:</b> All basic notebooks, report generation notebook<br>
<b>Dataset:</b> Breast Cancer (sklearn)
</div>

---

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- ✅ Understand DeepBridge's report system architecture
- ✅ Customize HTML templates for reports
- ✅ Add custom sections to reports
- ✅ Style reports with custom CSS
- ✅ Create branded reports for your organization
- ✅ Export reports in different formats

---

## 📚 Table of Contents

1. [Introduction](#intro)
2. [Setup](#setup)
3. [Report Architecture](#architecture)
4. [Generate Default Report](#default)
5. [Customize Report Metadata](#metadata)
6. [Custom CSS Styling](#css)
7. [Add Custom Sections](#sections)
8. [Brand Your Reports](#branding)
9. [Export Formats](#export)
10. [Conclusion](#conclusion)

<a id="intro"></a>
## 1. 📖 Introduction

### Why Customize Reports?

**The Need:**
- 🎨 **Branding** - Match your organization's visual identity
- 📊 **Custom metrics** - Add domain-specific KPIs
- 👥 **Stakeholder communication** - Tailor for different audiences
- 📋 **Compliance** - Meet regulatory reporting requirements
- 🔍 **Focus** - Highlight what matters most

**Real-world example:**
```python
# Default report: Generic validation results
exp.generate_report('report.html')  # Standard format

# Custom report: Branded, focused on fairness for audit
exp.generate_report(
    'audit_report.html',
    title='Q4 2024 Model Fairness Audit',
    logo='company_logo.png',
    custom_sections=['executive_summary', 'regulatory_compliance'],
    theme='corporate'
)  # Professional, audit-ready
```

### DeepBridge Report System

**Components:**

```
Report Generation Pipeline
├── Data Collection
│   ├── Test results
│   ├── Metrics
│   └── Visualizations
├── Template Engine
│   ├── Jinja2 templates
│   ├── CSS styling
│   └── JavaScript interactivity
├── Rendering
│   ├── HTML generation
│   ├── Asset embedding
│   └── Final assembly
└── Export
    ├── HTML (default)
    ├── PDF (via wkhtmltopdf)
    └── JSON (data only)
```

### Customization Levels

| Level | What You Can Do | Difficulty | Use Case |
|-------|-----------------|------------|----------|
| **Basic** | Change title, description | 🟢 Easy | Quick customization |
| **Intermediate** | Add CSS, modify colors | 🟡 Medium | Branding |
| **Advanced** | Custom templates, sections | 🔴 Hard | Full control |

**Let's explore customization!** 🎨

<a id="setup"></a>
## 2. 🛠️ Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pathlib import Path

# sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# DeepBridge
from deepbridge import DBDataset, Experiment

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Setup complete!")
print("📄 Topic: Report Customization")

### Prepare Experiment

In [ ]:
print("📊 Preparing experiment for report generation...\n")

# Load data
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

# Train model
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

# Create DBDataset
dataset = DBDataset(
    data=df,
    target_column='target',
    model=model,
    test_size=0.2,
    random_state=RANDOM_STATE
)

# Create Experiment
exp = Experiment(
    dataset=dataset,
    experiment_type='binary_classification',
    experiment_name='Breast Cancer Diagnosis Validation',
    random_state=RANDOM_STATE
)

print("✅ Experiment created")
print(f"   Name: {exp.experiment_name}")
print(f"   Type: {exp.experiment_type}")

### Run Tests for Reporting

In [ ]:
print("🔬 Running validation tests...\n")

# Run multiple tests to have content for the report
print("   Running robustness test...")
robustness_result = exp.run_test('robustness', config='quick')

print("   Running uncertainty test...")
uncertainty_result = exp.run_test('uncertainty', config='quick')

print("\n✅ Tests complete! Ready to generate reports.")

<a id="architecture"></a>
## 3. 🏗️ Report Architecture

### Understanding the Template System

DeepBridge uses **Jinja2** templates for report generation:

```html
<!-- Base template structure -->
<!DOCTYPE html>
<html>
<head>
    <title>{{ report_title }}</title>
    <style>
        /* CSS styles */
        {{ custom_css }}
    </style>
</head>
<body>
    <!-- Header -->
    <header>
        <img src="{{ logo_url }}" />
        <h1>{{ report_title }}</h1>
        <p>{{ description }}</p>
    </header>
    
    <!-- Summary Section -->
    <section id="summary">
        {{ summary_content }}
    </section>
    
    <!-- Test Results -->
    {% for test_result in test_results %}
    <section id="test-{{ test_result.name }}">
        {{ test_result.render() }}
    </section>
    {% endfor %}
    
    <!-- Custom Sections -->
    {% for section in custom_sections %}
    <section id="{{ section.id }}">
        {{ section.content }}
    </section>
    {% endfor %}
    
    <!-- Footer -->
    <footer>
        <p>Generated by DeepBridge</p>
        <p>{{ generation_date }}</p>
    </footer>
</body>
</html>
```

### Template Variables

| Variable | Type | Description | Example |
|----------|------|-------------|----------|
| `report_title` | str | Main report title | "Model Validation Report" |
| `description` | str | Report description | "Q4 2024 Audit" |
| `logo_url` | str | Company logo path | "logo.png" |
| `custom_css` | str | Custom CSS styles | "body { font-family: Arial; }" |
| `test_results` | list | Test result objects | `[robustness, fairness, ...]` |
| `custom_sections` | list | User-defined sections | `[{id, title, content}, ...]` |
| `generation_date` | datetime | Report creation time | "2024-11-05 14:30" |

<a id="default"></a>
## 4. 📋 Generate Default Report

In [ ]:
print("📋 Generating Default Report...\n")

# Generate basic report
output_path = 'reports/default_report.html'
Path('reports').mkdir(exist_ok=True)

try:
    exp.generate_report(output_path)
    print(f"✅ Default report generated: {output_path}")
    print(f"\n💡 This report uses DeepBridge's standard template")
    print(f"   • Standard layout")
    print(f"   • Default colors (blue theme)")
    print(f"   • All test results included")
    print(f"   • Basic metadata")
except Exception as e:
    print(f"Note: Report generation may require additional setup")
    print(f"   {str(e)[:100]}")
    print(f"\n💡 The concept remains: exp.generate_report() creates HTML reports")

<a id="metadata"></a>
## 5. 📝 Customize Report Metadata

### Add Custom Title and Description

In [ ]:
print("📝 Customizing Report Metadata...\n")

# Define custom metadata
metadata = {
    'title': 'Medical AI Validation Report - Q4 2024',
    'subtitle': 'Breast Cancer Diagnosis Model Audit',
    'description': '''
        Comprehensive validation report for the breast cancer diagnosis model.
        This report covers robustness, uncertainty quantification, and safety analysis.
        Prepared for FDA regulatory submission.
    ''',
    'organization': 'HealthTech AI Labs',
    'author': 'ML Validation Team',
    'version': '2.1.0',
    'date': '2024-11-05',
    'tags': ['medical', 'FDA', 'audit', 'safety-critical']
}

print("✅ Custom metadata defined:")
for key, value in metadata.items():
    if isinstance(value, str) and len(value) > 50:
        print(f"   {key}: {value[:50]}...")
    elif isinstance(value, list):
        print(f"   {key}: {', '.join(value)}")
    else:
        print(f"   {key}: {value}")

# Generate report with metadata
output_path = 'reports/custom_metadata_report.html'

try:
    exp.generate_report(
        output_path,
        title=metadata['title'],
        description=metadata['description'],
        metadata=metadata
    )
    print(f"\n✅ Custom metadata report generated: {output_path}")
except Exception as e:
    print(f"\n💡 Concept: Pass metadata to generate_report()")
    print(f"   exp.generate_report('report.html', title='...', metadata={{...}})")

<a id="css"></a>
## 6. 🎨 Custom CSS Styling

### Define Custom Styles

In [ ]:
print("🎨 Creating Custom CSS Styles...\n")

# Define custom CSS for corporate branding
custom_css = """
/* Corporate Theme - Professional Medical AI */

/* Color Palette */
:root {
    --primary-color: #1e3a5f;      /* Dark Blue */
    --secondary-color: #3a7ca5;    /* Medium Blue */
    --accent-color: #00a896;       /* Teal */
    --success-color: #52b788;      /* Green */
    --warning-color: #f4a261;      /* Orange */
    --danger-color: #e63946;       /* Red */
    --light-bg: #f8f9fa;           /* Light Gray */
    --text-color: #2d3748;         /* Dark Gray */
}

/* Typography */
body {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    color: var(--text-color);
    line-height: 1.6;
    background-color: var(--light-bg);
    margin: 0;
    padding: 0;
}

h1, h2, h3 {
    color: var(--primary-color);
    font-weight: 600;
}

h1 { font-size: 2.5rem; margin-bottom: 0.5rem; }
h2 { font-size: 2rem; border-bottom: 3px solid var(--accent-color); padding-bottom: 10px; }
h3 { font-size: 1.5rem; color: var(--secondary-color); }

/* Header */
header {
    background: linear-gradient(135deg, var(--primary-color), var(--secondary-color));
    color: white;
    padding: 40px 20px;
    text-align: center;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}

header h1 {
    color: white;
    margin: 0;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);
}

/* Content Container */
.container {
    max-width: 1200px;
    margin: 30px auto;
    padding: 20px;
    background: white;
    border-radius: 8px;
    box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
}

/* Sections */
section {
    margin-bottom: 40px;
    padding: 20px;
    border-left: 4px solid var(--accent-color);
    background: white;
}

/* Tables */
table {
    width: 100%;
    border-collapse: collapse;
    margin: 20px 0;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.05);
}

th {
    background: var(--primary-color);
    color: white;
    padding: 12px;
    text-align: left;
    font-weight: 600;
}

td {
    padding: 10px;
    border-bottom: 1px solid #e2e8f0;
}

tr:hover {
    background: var(--light-bg);
}

/* Status Badges */
.badge-success {
    background: var(--success-color);
    color: white;
    padding: 4px 12px;
    border-radius: 20px;
    font-size: 0.9em;
    font-weight: 600;
}

.badge-warning {
    background: var(--warning-color);
    color: white;
    padding: 4px 12px;
    border-radius: 20px;
    font-size: 0.9em;
    font-weight: 600;
}

.badge-danger {
    background: var(--danger-color);
    color: white;
    padding: 4px 12px;
    border-radius: 20px;
    font-size: 0.9em;
    font-weight: 600;
}

/* Footer */
footer {
    text-align: center;
    padding: 20px;
    background: var(--light-bg);
    color: var(--text-color);
    margin-top: 40px;
    border-top: 3px solid var(--accent-color);
}
"""

print("✅ Custom CSS defined with:")
print("   • Corporate color palette (blue/teal)")
print("   • Professional typography")
print("   • Styled tables and sections")
print("   • Status badges")
print(f"\n📏 CSS length: {len(custom_css)} characters")

### Generate Report with Custom CSS

In [ ]:
print("📄 Generating Styled Report...\n")

output_path = 'reports/styled_report.html'

try:
    exp.generate_report(
        output_path,
        title='Medical AI Validation Report',
        custom_css=custom_css
    )
    print(f"✅ Styled report generated: {output_path}")
    print(f"\n🎨 Report now features:")
    print(f"   • Corporate blue/teal color scheme")
    print(f"   • Professional typography")
    print(f"   • Enhanced visual hierarchy")
    print(f"   • Modern card-based layout")
except Exception as e:
    print(f"💡 Concept: Pass custom_css to generate_report()")
    print(f"   exp.generate_report('report.html', custom_css='...')")
    print(f"\n   Your CSS will override default styles!")

<a id="sections"></a>
## 7. ➕ Add Custom Sections

### Define Custom Content Sections

In [ ]:
print("➕ Creating Custom Report Sections...\n")

# Executive Summary
executive_summary = """
<div class="custom-section" id="executive-summary">
    <h2>📋 Executive Summary</h2>
    
    <div class="summary-box">
        <h3>Key Findings</h3>
        <ul>
            <li><span class="badge-success">✓</span> Model demonstrates high robustness across perturbations</li>
            <li><span class="badge-success">✓</span> Uncertainty calibration is excellent (ECE < 0.05)</li>
            <li><span class="badge-success">✓</span> No significant drift detected in recent data</li>
            <li><span class="badge-warning">⚠</span> Consider adding more adversarial robustness tests</li>
        </ul>
    </div>
    
    <div class="summary-box">
        <h3>Recommendations</h3>
        <p><strong>Deployment Status:</strong> <span class="badge-success">APPROVED</span></p>
        <p>The model meets all safety and performance criteria for production deployment.</p>
        <p><strong>Next Steps:</strong></p>
        <ol>
            <li>Monitor model performance weekly</li>
            <li>Set up automated drift detection</li>
            <li>Plan quarterly revalidation</li>
        </ol>
    </div>
</div>
"""

# Regulatory Compliance
regulatory_section = """
<div class="custom-section" id="regulatory">
    <h2>⚖️ Regulatory Compliance</h2>
    
    <table>
        <thead>
            <tr>
                <th>Requirement</th>
                <th>Standard</th>
                <th>Status</th>
                <th>Evidence</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>Model Validation</td>
                <td>FDA 21 CFR Part 11</td>
                <td><span class="badge-success">PASS</span></td>
                <td>Section 3.1</td>
            </tr>
            <tr>
                <td>Robustness Testing</td>
                <td>ISO 13485</td>
                <td><span class="badge-success">PASS</span></td>
                <td>Section 3.2</td>
            </tr>
            <tr>
                <td>Uncertainty Quantification</td>
                <td>HIPAA Security Rule</td>
                <td><span class="badge-success">PASS</span></td>
                <td>Section 3.3</td>
            </tr>
            <tr>
                <td>Adversarial Robustness</td>
                <td>Best Practice</td>
                <td><span class="badge-warning">PARTIAL</span></td>
                <td>Needs improvement</td>
            </tr>
        </tbody>
    </table>
    
    <p><strong>Compliance Score:</strong> 95% (38/40 criteria met)</p>
</div>
"""

# Risk Assessment
risk_section = """
<div class="custom-section" id="risk">
    <h2>⚠️ Risk Assessment</h2>
    
    <table>
        <thead>
            <tr>
                <th>Risk Category</th>
                <th>Likelihood</th>
                <th>Impact</th>
                <th>Overall Risk</th>
                <th>Mitigation</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>False Negatives (missed cancer)</td>
                <td>Low</td>
                <td>Critical</td>
                <td><span class="badge-warning">MEDIUM</span></td>
                <td>High recall threshold, human review</td>
            </tr>
            <tr>
                <td>Data Drift</td>
                <td>Medium</td>
                <td>High</td>
                <td><span class="badge-warning">MEDIUM</span></td>
                <td>Automated monitoring, quarterly retraining</td>
            </tr>
            <tr>
                <td>Adversarial Attacks</td>
                <td>Low</td>
                <td>Medium</td>
                <td><span class="badge-success">LOW</span></td>
                <td>Input validation, anomaly detection</td>
            </tr>
        </tbody>
    </table>
</div>
"""

custom_sections = [
    {'id': 'executive-summary', 'title': 'Executive Summary', 'content': executive_summary},
    {'id': 'regulatory', 'title': 'Regulatory Compliance', 'content': regulatory_section},
    {'id': 'risk', 'title': 'Risk Assessment', 'content': risk_section}
]

print("✅ Custom sections defined:")
for section in custom_sections:
    print(f"   • {section['title']} ({len(section['content'])} chars)")

### Generate Report with Custom Sections

In [ ]:
print("📄 Generating Full Custom Report...\n")

output_path = 'reports/full_custom_report.html'

try:
    exp.generate_report(
        output_path,
        title='Medical AI Validation Report - Complete',
        description='Comprehensive validation with custom sections',
        custom_css=custom_css,
        custom_sections=custom_sections,
        metadata=metadata
    )
    print(f"✅ Full custom report generated: {output_path}")
    print(f"\n📋 Report includes:")
    print(f"   • Custom metadata")
    print(f"   • Corporate styling (CSS)")
    print(f"   • Executive summary")
    print(f"   • Regulatory compliance section")
    print(f"   • Risk assessment")
    print(f"   • Standard test results")
except Exception as e:
    print(f"💡 Concept: Combine all customizations")
    print(f"""   exp.generate_report(
       'report.html',
       title='...',
       custom_css='...',
       custom_sections=[...],
       metadata={{...}}
   )""")

<a id="branding"></a>
## 8. 🏢 Brand Your Reports

### Add Company Logo and Branding

In [ ]:
print("🏢 Creating Branded Report...\n")

branding = {
    'logo_url': 'https://example.com/company_logo.png',
    'company_name': 'HealthTech AI Labs',
    'primary_color': '#1e3a5f',
    'secondary_color': '#00a896',
    'font_family': 'Segoe UI, sans-serif',
    'footer_text': 'Confidential - HealthTech AI Labs © 2024'
}

print("✅ Branding configuration:")
for key, value in branding.items():
    print(f"   {key}: {value}")

print("\n💡 Branding Tips:")
print("   • Use your company's color palette")
print("   • Add logo to header (< 200px height)")
print("   • Match font to corporate guidelines")
print("   • Include confidentiality notice")
print("   • Add version/date stamps")
print("\n🎨 Consistent branding builds trust with stakeholders!")

<a id="export"></a>
## 9. 💾 Export Formats

### Export to Different Formats

In [ ]:
print("💾 Report Export Formats\n")
print("=" * 70)

export_options = pd.DataFrame({
    'Format': ['HTML', 'PDF', 'JSON', 'Markdown'],
    'Use Case': [
        'Interactive viewing, sharing',
        'Archival, regulatory submission',
        'Programmatic access, APIs',
        'Documentation, wikis'
    ],
    'Pros': [
        'Interactive, embeds assets',
        'Professional, printable',
        'Machine-readable, flexible',
        'Human-readable, versionable'
    ],
    'Cons': [
        'Not printable',
        'Requires conversion tool',
        'No formatting',
        'Limited styling'
    ],
    'Command': [
        'generate_report("r.html")',
        'generate_report("r.pdf")',
        'export_json("r.json")',
        'export_markdown("r.md")'
    ]
})

display(export_options.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap'
}))

print("\n💡 Recommendations:")
print("   • Development: HTML (fastest iteration)")
print("   • Stakeholders: HTML (interactive, visual)")
print("   • Audit/Archive: PDF (official record)")
print("   • Automation: JSON (programmatic access)")

<a id="conclusion"></a>
## 10. 🎓 Conclusion

### What You Learned

- ✅ **Report architecture** - Template system, components
- ✅ **Metadata customization** - Titles, descriptions, tags
- ✅ **CSS styling** - Custom colors, fonts, layouts
- ✅ **Custom sections** - Add domain-specific content
- ✅ **Branding** - Company logos, corporate identity
- ✅ **Export formats** - HTML, PDF, JSON, Markdown

### Key Takeaways

1. 🎨 **Start simple** - Customize metadata first
2. 🏢 **Brand consistently** - Use corporate colors/fonts
3. 📋 **Add value** - Custom sections for stakeholders
4. ⚖️ **Meet requirements** - Regulatory compliance sections
5. 💾 **Choose format** - HTML for viewing, PDF for archives
6. 🔄 **Iterate** - Refine based on feedback

### Customization Hierarchy

```
🔵 LEVEL 1: Basic (5 minutes)
└── Change title and description

🟡 LEVEL 2: Intermediate (30 minutes)
├── Add custom CSS for branding
└── Include company logo

🔴 LEVEL 3: Advanced (2-4 hours)
├── Create custom sections
├── Add regulatory compliance content
├── Risk assessment tables
└── Full template customization
```

### Example: Complete Custom Report

```python
# Production-ready custom report
exp.generate_report(
    'audit_report_2024_q4.html',
    
    # Basic metadata
    title='Q4 2024 Model Audit Report',
    description='Comprehensive validation for FDA submission',
    
    # Branding
    logo_url='company_logo.png',
    custom_css=corporate_css,
    
    # Custom content
    custom_sections=[
        executive_summary,
        regulatory_compliance,
        risk_assessment,
        recommendations
    ],
    
    # Metadata
    metadata={
        'organization': 'HealthTech AI',
        'author': 'ML Validation Team',
        'version': '2.1.0',
        'confidential': True,
        'tags': ['FDA', 'audit', 'Q4']
    }
)
```

### Next Steps

1. **Create your template** - Start with defaults, customize gradually
2. **Test with stakeholders** - Get feedback early
3. **Standardize** - Create reusable templates
4. **Automate** - Generate reports in CI/CD
5. **Iterate** - Improve based on usage

---

**Remember: Great reports communicate insights effectively to all stakeholders!** 📄✨